In [21]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
from datetime import datetime, timezone
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.dataset_pytorch import DatasetPytorch
from modeling.trainer import Trainer
from modeling.evaluate import evaluate_lgb_regressor, evaluate_torch_regressor, evaluate_torch_regressor_multiasset
from observability.mlflow_integration import log_experiment

from config.experiments.cur_experiment import config


In [11]:
retriever = AlpacaMarketsRetriever()

retrieval_result = retriever.bars(
    symbol_or_symbols=config.data_config.symbol_or_symbols, 
    start=config.data_config.start, 
    end=config.data_config.end)

In [3]:
dataset_creator = DatasetCreator(
    features=config.data_config.features,
    target=config.data_config.target,
    normalizer=config.data_config.normalizer,
    missing_values_handler=config.data_config.missing_values_handler,
    train_set_last_date=config.data_config.train_set_last_date, 
    in_seq_len=config.data_config.in_seq_len,
    multi_asset_prediction=config.data_config.multi_asset_prediction,
)

X_train, y_train, next_return_train, X_test, y_test, next_return_test = dataset_creator.create_dataset_numpy(retrieval_result)
X_train.shape, y_train.shape, next_return_train.shape, X_test.shape, y_test.shape, next_return_test.shape

2025-07-01 19:16:43,107 - INFO - Processing AAPL …
2025-07-01 19:16:43,410 - INFO - Imputing 496 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:45,213 - INFO - Processing ADBE …
2025-07-01 19:16:45,441 - INFO - Imputing 5392 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:47,202 - INFO - Processing ADI …
2025-07-01 19:16:47,436 - INFO - Imputing 6204 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:49,265 - INFO - Processing AMAT …
2025-07-01 19:16:49,497 - INFO - Imputing 4035 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:51,264 - INFO - Processing AMD …
2025-07-01 19:16:51,530 - INFO - Imputing 214 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:53,283 - INFO - Processing ANET …
2025-07-01 19:16:53,514 - INFO - Imputing 5097 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:55,262 - INFO - Processing AVGO …
2025-07-01 19:16:55,525 - INFO - Imputing 1059 NaN rows out of 97359 with forward fill..
2025-07-01 19:16:57,279 

((49, 79999, 30, 37),
 (49, 79999),
 (49, 79999),
 (49, 7341, 30, 37),
 (49, 7341),
 (49, 7341))

In [4]:
if config.data_config.multi_asset_prediction:
    X_train = np.swapaxes(X_train, 0, 1)
    y_train = np.swapaxes(y_train, 0, 1)
    next_return_train = np.swapaxes(next_return_train, 0, 1)

    X_test = np.swapaxes(X_test, 0, 1)
    y_test = np.swapaxes(y_test, 0, 1)
    next_return_test = np.swapaxes(next_return_test, 0, 1)

X_train.shape, y_train.shape, next_return_train.shape, X_test.shape, y_test.shape, next_return_test.shape

((79999, 49, 30, 37),
 (79999, 49),
 (79999, 49),
 (7341, 49, 30, 37),
 (7341, 49),
 (7341, 49))

In [5]:
y_train.mean(), y_test.mean()

(0.50199676, 0.5025806)

In [6]:
train_loader = DatasetPytorch(X_train, y_train, learning_task='regression').as_dataloader(
    batch_size=config.data_config.batch_size,
    shuffle=config.data_config.shuffle,
)
test_loader = DatasetPytorch(X_test, y_test, learning_task='regression').as_dataloader(
    batch_size=config.data_config.batch_size,
    shuffle=config.data_config.shuffle
)

In [12]:
model = config.model_config.model
model

TemporalSpatial(
  (asset_embed): Embedding(49, 16)
  (asset_proj): Linear(in_features=16, out_features=128, bias=False)
  (lstm): LSTM(37, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (spatial_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [13]:
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    loss_fn=config.train_config.loss_fn,
    optimizer=config.train_config.optimizer,
    scheduler=config.train_config.scheduler,
    num_epochs=config.train_config.num_epochs,
    device=config.train_config.device,
    metrics=config.train_config.metrics,
    save_path=config.train_config.save_path
)

In [14]:
model, history = trainer.train()

2025-07-01 19:21:22,857 - INFO - Epoch 1/1
2025-07-01 19:22:24,968 - INFO - Train Loss: 0.1314          
2025-07-01 19:22:24,968 - INFO - Train Rmse: 0.3615
2025-07-01 19:22:24,968 - INFO - Val   Loss: 0.1104
2025-07-01 19:22:24,969 - INFO - Val   Rmse: 0.3320
2025-07-01 19:22:24,969 - INFO - 


In [22]:
evaluate_torch_regressor_multiasset(model, X_train, y_train, X_test, y_test, next_return_test)

torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Size([1024, 49, 30, 37])
torch.Si

In [20]:
log_experiment(
    config=config, 
    model=model, 
    history=history,
    input_data_sample=next(iter(train_loader))[0].to(trainer.device))

2025/06/26 15:35:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LSTM Default' already exists. Creating a new version of this model...
2025/06/26 15:35:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LSTM Default, version 10


🏃 View run gentle-loon-699 at: http://127.0.0.1:8080/#/experiments/439216085822475480/runs/54deb1104660468d9ffb4e7e278e9cfb
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/439216085822475480


Created version '10' of model 'LSTM Default'.


In [10]:
evaluate_lgb_regressor(X_train, y_train, X_test, y_test, next_return_test)

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9435
[LightGBM] [Info] Number of data points in the train set: 7371, number of used features: 37
[LightGBM] [Info] Start training from score 0.497863
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
